**Introduction**
This notebook shows how to load and evaluate the MNIST and model synthesized and trained as described in the following paper:
M.Sinn, M.Wistube, B.Buesser, M.Nicolae,M.N.Tran:**Evolutionary Search for Adversarially Robust Neural Network **ICLR SafeML Workshop

In [1]:
!pip install adversarial_robustness_toolbox


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.9 MB/s eta 0:00:00


In [20]:
import warnings
warnings.filterwarnings('ignore')
from keras.datasets import mnist
from keras.models import load_model
from keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from keras.losses import CategoricalCrossentropy
import numpy as np

from art import config
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import ProjectedGradientDescent
from art.utils import get_file


**MNIST**
Three different MNIST models are available. Use the following URLs to access them:

mnist_ratio=0.h5: trained on 100% benign samples
 (https://www.dropbox.com/s/bv1xwjaf1ov4u7y/mnist_ratio%3D0.h5?dl=1)
mnist_ratio=0.5.h5: trained on 50% benign and 50% adversarial samples
(https://www.dropbox.com/s/0skvoxjd6klvti3/mnist_ratio%3D0.5.h5?dl=1)
mnist_ratio=1.h5: trained on 100% adversarial samples
 (https://www.dropbox.com/s/oa2kowq7kgaxh1o/mnist_ratio%3D1.h5?dl=1)

Load Data:

In [21]:

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')/255
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

E.g. Load the model trained on 50% benign and 50% adversarial samples:

In [23]:
  path = get_file('mnist_ratio=0.5.h5',extract=False, path=config.ART_DATA_PATH,
                url='https://www.dropbox.com/s/0skvoxjd6klvti3/mnist_ratio%3D0.5.h5?dl=1')
model = load_model(path)
loss_object = CategoricalCrossentropy()
classifier = TensorFlowV2Classifier(clip_values=(0, 1), model=model, nb_classes=10, input_shape=(28, 28, 1), loss_object=loss_object)

#classifier = TensorFlowV2Classifier(model=model, use_logits=False, clip_values=[0,1])

Assess accuracy on first n benign test sampes :

In [24]:
n = 10000
y_pred = classifier.predict(X_test[:n])
accuracy = np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_test[:n], axis=1))
print("Accuracy on first %i benign test samples: %f" % (n, accuracy))

Accuracy on first 10000 benign test samples: 0.995100


Define Adversarial Attack

In [25]:
attack = ProjectedGradientDescent(classifier, eps=0.3, eps_step=0.01, max_iter=40, targeted=False,
                                  num_random_init=True)

Assess accuracy on first n adversarial test samples:

In [27]:
n = 10
X_test_adv = attack.generate(X_test[:n], y=y_test[:n])
y_adv_pred = classifier.predict(X_test_adv)
accuracy = np.mean(np.argmax(y_adv_pred, axis=1) == np.argmax(y_test[:n], axis=1))
print("Accuracy on first %i adversarial test samples: %f" % (n, accuracy))

PGD - Batches: 0it [00:00, ?it/s]

Accuracy on first 10 adversarial test samples: 1.000000
